In [1]:
import pandas as pd
import json
import os
from IPython.display import display

import pywikibot
import requests
from tqdm import tqdm

In [2]:
# Data source: http://www.cs.cmu.edu/~ark/personas/

# Define the paths
DATA_PATH = 'data/'
CLEAN_DATA_PATH = 'clean_data/'

# 1. Reformat the data to have a clean Database

In [3]:
# We do not use the name.clusters.txt file as it is quite useless for our purpose

In [4]:
# Separate the dict columns for the movies table

# Load the data
movies_df = pd.read_csv(DATA_PATH + 'movie.metadata.tsv', delimiter='\t', header=None, names=['wiki_id', 'id', 'name', 'release_date', 'revenue', 'runtime', 'languages', 'countries', 'genres'])
display(movies_df.head())

# Separate the languages to another table
movies_df["languages"] = movies_df["languages"].apply(lambda x: list(json.loads(x).values()))
movies_languages_df = movies_df[["id", "languages"]].explode("languages").reset_index(drop=True)
movies_languages_df = movies_languages_df.rename(columns={"languages": "language", "id": "movie_id"})
display(movies_languages_df.head())

# Separate the genres to another table
movies_df["genres"] = movies_df["genres"].apply(lambda x: list(json.loads(x).values()))
movies_genres_df = movies_df[["id", "genres"]].explode("genres").reset_index(drop=True)
movies_genres_df = movies_genres_df.rename(columns={"genres": "genre", "id": "movie_id"})
display(movies_genres_df.head())

# Separate the countries played in to another table
movies_df["countries"] = movies_df["countries"].apply(lambda x: list(json.loads(x).values()))
movies_countries_playedin_df = movies_df[["id", "countries"]].explode("countries").reset_index(drop=True)
movies_countries_playedin_df = movies_countries_playedin_df.rename(columns={"countries": "country", "id": "movie_id"})
display(movies_countries_playedin_df.head())

,wiki_id,id,name,release_date,revenue,runtime,languages,countries,genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


,movie_id,language
0,/m/03vyhn,English Language
1,/m/08yl5d,English Language
2,/m/0crgdbh,Norwegian Language
3,/m/0285_cd,English Language
4,/m/01mrr1,German Language


,movie_id,genre
0,/m/03vyhn,Thriller
1,/m/03vyhn,Science Fiction
2,/m/03vyhn,Horror
3,/m/03vyhn,Adventure
4,/m/03vyhn,Supernatural


,movie_id,country
0,/m/03vyhn,United States of America
1,/m/08yl5d,United States of America
2,/m/0crgdbh,Norway
3,/m/0285_cd,United Kingdom
4,/m/01mrr1,Germany


In [5]:
# Add a new column to the movies for the area
areas = {
	"Northen America": set(["United States of America", "Canada"]),
	"Europe": set(["United Kingdom", "France", "Italy", "Germany", "Spain", "West Germany", "Belgium", "German Democratic Republic", "Ireland", "Switzerland", "Austria", "England", "Luxembourg", "Portugal"]),
	"Asia": set(["Japan", "Hong Kong", "China", "South Korea", "Taiwan"]),
	"India": set(["India", "Pakistan"])
}
country_to_area = {country: area for area, countries in areas.items() for country in countries}

# Set default area to None
movies_df["area"] = None

In [6]:
def get_area(country):
	if country in country_to_area:
		return country_to_area[country]
	else:
		return "Other"

# Set the areas if the number of countries played in is 1
movies_with_one_country_df = movies_df[movies_df["countries"].apply(lambda x: len(x) == 1)]
print(f"Fetching areas for {len(movies_with_one_country_df)} movies")
movies_df.loc[movies_with_one_country_df.index, "area"] = movies_with_one_country_df["countries"].apply(lambda x: get_area(x[0]))
display(movies_df.head())

Fetching areas for 64369 movies


,wiki_id,id,name,release_date,revenue,runtime,languages,countries,genres,area
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",Northen America
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",Northen America
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",Other
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",Europe
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama],Europe


In [7]:
def get_area_from_unanimity(countries):
	area = None
	for country in countries:
		if country in country_to_area:
			if area is None:
				area = country_to_area[country]
			elif area != country_to_area[country]:
				return None
		else:
			return None
	return area

# For each remaining movie (only the ones with None as area), if the countries played in are in the same area, set the area to that area
movies_without_area_df = movies_df[movies_df["area"].isnull()]
print(f"Combining areas for {len(movies_without_area_df)} movies")
movies_df.loc[movies_without_area_df.index, "area"] = movies_without_area_df["countries"].apply(lambda x: get_area_from_unanimity(x))
display(movies_df.head())

Combining areas for 17372 movies


,wiki_id,id,name,release_date,revenue,runtime,languages,countries,genres,area
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,[English Language],[United States of America],"[Thriller, Science Fiction, Horror, Adventure,...",Northen America
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,[English Language],[United States of America],"[Mystery, Biographical film, Drama, Crime Drama]",Northen America
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,[Norwegian Language],[Norway],"[Crime Fiction, Drama]",Other
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,[English Language],[United Kingdom],"[Thriller, Erotic thriller, Psychological thri...",Europe
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,[German Language],[Germany],[Drama],Europe


In [8]:
wikipedia = pywikibot.Site('en', 'wikipedia')
country_property = 'P495'

def get_area_from_wikipedia(movie_wiki_id):
	try:
		wiki_api_url = f'https://en.wikipedia.org/w/api.php?action=query&pageids={movie_wiki_id}&format=json'
		json_data = requests.get(wiki_api_url).json()
		movie_title = json_data["query"]["pages"][str(movie_wiki_id)]["title"]
		movie_page = pywikibot.Page(wikipedia, movie_title)
		movie_item = pywikibot.ItemPage.fromPage(movie_page)
		movie_item_dict = movie_item.get()
		movie_country = movie_item_dict["claims"][country_property][0].getTarget().labels["en"]
		return country_to_area[movie_country] if movie_country in country_to_area else "Other"
	except:
		return "Not found"
	
# For each remaining movie (that had the number of countries played in != 1), we set the area retrieved from wikipedia by scraping
movies_without_area = movies_df[movies_df["area"].isnull()]
print(f"Retriving areas for {len(movies_without_area)} movies")
movies_df.loc[movies_without_area.index, "area"] = movies_without_area["wiki_id"].apply(lambda x: get_area_from_wikipedia(x))

Retriving areas for 14423 movies


ERROR: Traceback (most recent call last):
  File "/home/reezer/Documents/epfl/master-1/CS401 - Applied Data Analysis/Project/venv/lib64/python3.11/site-packages/pywikibot/data/api/_requests.py", line 682, in _http_request
    response = http.request(self.site, uri=uri,
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/reezer/Documents/epfl/master-1/CS401 - Applied Data Analysis/Project/venv/lib64/python3.11/site-packages/pywikibot/comms/http.py", line 283, in request
    r = fetch(baseuri, headers=headers, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/reezer/Documents/epfl/master-1/CS401 - Applied Data Analysis/Project/venv/lib64/python3.11/site-packages/pywikibot/comms/http.py", line 457, in fetch
    callback(response)
  File "/home/reezer/Documents/epfl/master-1/CS401 - Applied Data Analysis/Project/venv/lib64/python3.11/site-packages/pywikibot/comms/http.py", line 333, in error_handling_callback
    raise ServerError(response)
pywikibot.excep

In [9]:
# Drop the useless remaining columns from the movies_df
movies_df = movies_df.drop(columns=["languages", "genres", "countries"])
display(movies_df.head())

,wiki_id,id,name,release_date,revenue,runtime,area
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,Northen America
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,Northen America
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,Other
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,Europe
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,Europe


In [10]:
# Store the summaries in a separate table, using the movie_id instead of the wiki_id
# We could have merged it with the movies table, but we want to keep the summaries in a separate table because it is a very large text field

# Load the data
movies_summaries_df = pd.read_csv(DATA_PATH + 'plot_summaries.txt', delimiter='\t', header=None, names=['wiki_id', 'summary'])
display(movies_summaries_df.head())

# Merge the summaries with the movies table and only keep the movie_id and summary columns
movies_summaries_df = movies_summaries_df.merge(movies_df, on="wiki_id")[["id", "summary"]]
movies_summaries_df = movies_summaries_df.rename(columns={"id": "movie_id"})
display(movies_summaries_df.head())

,wiki_id,summary
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...


,movie_id,summary
0,/m/076w2lb,"Shlykov, a hard-working taxi driver and Lyosha..."
1,/m/0gkz15s,The nation of Panem consists of a wealthy Capi...
2,/m/051zjwb,Poovalli Induchoodan is sentenced for six yea...
3,/m/06xtz3,"The Lemon Drop Kid , a New York City swindler,..."
4,/m/02tqm5,Seventh-day Adventist Church pastor Michael Ch...


In [11]:
# Store the characters in a separate table

# Load the data
characters_df = pd.read_csv(DATA_PATH + 'character.metadata.tsv', delimiter='\t', header=None, names=['movie_wiki_id', 'movie_id', 'movie_release_date', 'name', 'actor_birth_date', 'actor_gender', 'actor_height','actor_ethinicity_id', 'actor_name', 'actor_age', 'actor_map_id', 'id', 'actor_id']) 
display(characters_df.head())

,movie_wiki_id,movie_id,movie_release_date,name,actor_birth_date,actor_gender,actor_height,actor_ethinicity_id,actor_name,actor_age,actor_map_id,id,actor_id
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


In [12]:
# Clean the characters categories table

# Load the data
characters_categories_df = pd.read_csv(DATA_PATH + 'tvtropes.clusters.txt', delimiter='\t', header=None, names=['category', 'meta'])
display(characters_categories_df.head())

# Fetch the characters ids from the meta column, and remove the useless columns
characters_categories_df["meta"] = characters_categories_df["meta"].apply(lambda x: json.loads(x))
characters_categories_df["character_id"] = characters_categories_df["meta"].apply(lambda x: x["id"])
characters_categories_df = characters_categories_df.drop(columns=["meta"])
display(characters_categories_df.head())

,category,meta
0,absent_minded_professor,"{""char"": ""Professor Philip Brainard"", ""movie"":..."
1,absent_minded_professor,"{""char"": ""Professor Keenbean"", ""movie"": ""Richi..."
2,absent_minded_professor,"{""char"": ""Dr. Reinhardt Lane"", ""movie"": ""The S..."
3,absent_minded_professor,"{""char"": ""Dr. Harold Medford"", ""movie"": ""Them!..."
4,absent_minded_professor,"{""char"": ""Daniel Jackson"", ""movie"": ""Stargate""..."


,category,character_id
0,absent_minded_professor,/m/0jy9q0
1,absent_minded_professor,/m/02vchl3
2,absent_minded_professor,/m/0k6fkc
3,absent_minded_professor,/m/0k6_br
4,absent_minded_professor,/m/0k3rhh


In [13]:
# Remove all the previous files from the clean_data folder
for filename in os.listdir(CLEAN_DATA_PATH):
	os.remove(CLEAN_DATA_PATH + filename)

# Save the data
movies_df.to_csv(CLEAN_DATA_PATH + 'movies.csv', index=False)
movies_languages_df.to_csv(CLEAN_DATA_PATH + 'movies_languages.csv', index=False)
movies_genres_df.to_csv(CLEAN_DATA_PATH + 'movies_genres.csv', index=False)
movies_countries_playedin_df.to_csv(CLEAN_DATA_PATH + 'movies_countries_playedin.csv', index=False)
movies_summaries_df.to_csv(CLEAN_DATA_PATH + 'movies_summaries.csv', index=False)
characters_df.to_csv(CLEAN_DATA_PATH + 'characters.csv', index=False)
characters_categories_df.to_csv(CLEAN_DATA_PATH + 'characters_categories.csv', index=False)